### 상대모멘텀 
- 10개의 주식 데이터를 이용하여 투자 전략 테스팅
1. 월초부터 월말까지의 수정주가를 이용하여 월별 수익율
2. 월별 수익율이 높은 순서중 n개의 주식을 선택 
3. 해당하는 주식들을 매수 후 매도하여 수익율 계산

In [ ]:
import pandas as pd 
from datetime import datetime
import numpy as np
import os 
import glob
import warnings

In [ ]:
# warning message 제거 
warnings.filterwarnings('ignore')

In [14]:
# 월별 수익율을 계산하는 함수 생성 
def create_1M_rtn(_df, _ticker, _start = "2010-01-01", _col = 'Adj Close'):
    # 복사본 생성 
    result = _df.copy()
    # 컬럼에 Date가 포함되어있는지 확인 
    if 'Date' in result.columns:
        result = result.loc[result['Date'] >= _start, ['Date', _col]]
        # Date 컬럼의 데이터를 시계열로 변경
        result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d')
        result.set_index('Date', inplace=True)
    else:
        result.index = pd.to_datetime(result.index, inplace=True)
        result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    # 기준 년월의 중복데이터를 제거하고 고유한 값들을 리스트로 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [ ]:
aapl = pd.read_csv('../../csv/AAPL.csv')

In [16]:
sample_aapl, ym_list = create_1M_rtn(aapl, 'AAPL')

In [17]:
sample_aapl.head()

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL


In [ ]:
ym_list

In [18]:
# 특정 경로에 있는 파일들의 목록을 로드 
os.listdir("./data")

files = glob.glob("./data/*.csv")

# 새로운 데이터프레임을 생성 
# 종목별 데이터프레임 
stock_df = pd.DataFrame()
# 월말 데이터프레임 
month_last_df = pd.DataFrame()

for file in files:
    folder, name = os.path.split(file)
    # print(folder, name)
    head, tail = os.path.splitext(name)
    # print(head, tail)
    # head는 create_1M_rtn 함수에 ticker 인자값으로 사용

    # 데이터 파일을 로드 
    read_df = pd.read_csv(file)

    # create_1M_rtn 함수를 호출 
    price_df, ym_list = create_1M_rtn(read_df, head)

    # 유니언 결합 (단순한 행 결합 함수)
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 월별 상태 모멘텀을 계산하기 위해 1개월간의 수익율 계산
    for ym in ym_list:
        flag = price_df['STD-YM'] == ym
        m_rtn = price_df.loc[flag,].iloc[-1, 0] / price_df.loc[flag,].iloc[0, 0]
        price_df.loc[flag, '1m_rtn'] = m_rtn
        data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat([month_last_df, data], axis=0)

In [19]:
month_last_df.head()

,CODE,1m_rtn
Date,,
2010-01-29,AAPL,0.897435
2010-02-26,AAPL,1.050789
2010-03-31,AAPL,1.124456
2010-04-30,AAPL,1.106454
2010-05-28,AAPL,0.964445
